In [2]:
#Importando as bibliotecas
import re
import nltk
import random
import numpy as np
import pandas as pd

In [3]:
#Lendo o dataset tratado
df = pd.read_excel('C:/Users/atmaz1/Desktop/AED - Classificação de Publicações/bdpublicacaofinal.xlsx')
df['classe'].value_counts()

classe
Sentenca                 9511
Peticao Intermediaria    9178
Prazo Parte Contraria    5293
Audiencia                4378
Acordao                  4238
Pauta de Julgamento      3221
Contrarrazoes            3162
Outros                   2897
Conclusao                1988
Contestacao              1192
Liminar                   422
Decisao                    98
Name: count, dtype: int64

In [3]:
pip install --user -U nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk

In [5]:
#@title Carregando Bibliotecas do nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\atmaz1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\atmaz1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
# Classes desejadas e número de linhas por classe
classes_desejadas = ['Acordao', 'Audiencia', 'Sentenca']
num_linhas_por_classe = 4000

# Lista para armazenar amostras de cada classe
amostras_por_classe = []

# Para cada classe desejada, selecionar aleatoriamente 'num_linhas_por_classe' amostras
for classe in classes_desejadas:
    df_classe = df[df['classe'] == classe]
    
    # Verificar se a classe tem linhas suficientes
    if len(df_classe) < num_linhas_por_classe:
        raise ValueError(f"A classe {classe} não tem linhas suficientes. Tem apenas {len(df_classe)} linhas.")
    
    # Selecionar aleatoriamente 'num_linhas_por_classe' amostras
    amostras_classe = df_classe.sample(n=num_linhas_por_classe, random_state=42)
    amostras_por_classe.append(amostras_classe)

# Combinar todas as amostras em um único DataFrame
df_amostras = pd.concat(amostras_por_classe)

# Verificar a distribuição das classes nas amostras selecionadas
print(df_amostras['classe'].value_counts())

# Salvar as amostras selecionadas em um novo arquivo CSV, se necessário
df_amostras.to_csv('amostras_selecionadas.csv', index=False)


classe
Acordao      4000
Audiencia    4000
Sentenca     4000
Name: count, dtype: int64


In [4]:
dataframe = pd.read_csv('amostras_selecionadas.csv')

In [5]:
#Limpeza no texto
def process(x):
    x = re.sub('[,\.!?:()"]', '', x) # remoção de caracteres especiais
    x = re.sub('<.*?>', ' ', x)      # remoção de conteudos entre <>
    x = re.sub('http\S+', ' ', x)     # remoção do https
    x = re.sub('[^a-zA-Z0-9]', ' ', x)  # remoção de qualquer caractere que não seja uma letra ou números entre 0 e 9
    x = re.sub('\s+', ' ', x)            # remoção de espaços em branco duplicados
    return x.lower().strip()

dataframe['texto'] = dataframe['descricao_classificacoes'].apply(lambda x: process(x))

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\atmaz1\AppData\Local\Temp\2\ipykernel_6304\3529497774.py:3: SyntaxWarning: invalid escape sequence '\.'
  x = re.sub('[,\.!?:()"]', '', x) # remoção de caracteres especiais
C:\Users\atmaz1\AppData\Local\Temp\2\ipykernel_6304\3529497774.py:5: SyntaxWarning: invalid escape sequence '\S'
  x = re.sub('http\S+', ' ', x)     # remoção do https
C:\Users\atmaz1\AppData\Local\Temp\2\ipykernel_6304\3529497774.py:7: SyntaxWarning: invalid escape sequence '\s'
  x = re.sub('\s+', ' ', x)            # remoção de espaços em branco duplicados


In [6]:
dataframe = dataframe.drop('descricao_classificacoes', axis=1)

In [7]:
# Remoção de Stop Words
sw_set = set(nltk.corpus.stopwords.words('portuguese'))
print(sw_set)

def sw_remove(x):
    words = nltk.tokenize.word_tokenize(x)
    filtered_list = [word for word in words if word not in sw_set]
    return ' '.join(filtered_list)

dataframe['texto_preprocessing'] = dataframe['texto'].apply(lambda x: sw_remove(x))

{'seria', 'tivemos', 'as', 'deles', 'é', 'temos', 'mesmo', 'fossem', 'minha', 'tivesse', 'dos', 'havemos', 'isto', 'aquele', 'tuas', 'essas', 'terei', 'um', 'estamos', 'delas', 'te', 'nas', 'quem', 'entre', 'teria', 'aqueles', 'era', 'haja', 'tivessem', 'teremos', 'tem', 'em', 'tenhamos', 'aquilo', 'me', 'tivéramos', 'teve', 'a', 'hão', 'estivermos', 'isso', 'estiverem', 'lhes', 'houverá', 'serei', 'houve', 'os', 'estão', 'aos', 'elas', 'houvemos', 'estiver', 'lhe', 'fora', 'estejam', 'pelo', 'sou', 'suas', 'estar', 'tenho', 'depois', 'quando', 'eram', 'houvéramos', 'houverem', 'tiverem', 'no', 'à', 'teríamos', 'nossa', 'ao', 'tu', 'seriam', 'tinham', 'estive', 'por', 'terão', 'ser', 'dela', 'estivera', 'houveremos', 'tínhamos', 'como', 'houvera', 'houverão', 'se', 'houvéssemos', 'sejamos', 'houveríamos', 'aquelas', 'às', 'esteve', 'nos', 'das', 'até', 'e', 'já', 'forem', 'houveriam', 'o', 'estava', 'teu', 'estivéssemos', 'terá', 'esse', 'teus', 'tivera', 'que', 'tivéssemos', 'fomos', 

In [8]:
from sklearn.model_selection import train_test_split
#Separando os Dados de Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(dataframe['texto_preprocessing'], dataframe['classe'], test_size=0.1, random_state=1)

print('\033[1m' + 'train_txt.shape:' + '\033[0m', X_train.shape)
print('\033[1m' + 'test_txt.shape:' + '\033[0m', X_test.shape)
print('\033[1m' + 'train_class.shape:' + '\033[0m', y_train.shape)
print('\033[1m' + 'test_class.shape:' + '\033[0m', y_test.shape)

train_txt.shape: (10800,)
test_txt.shape: (1200,)
train_class.shape: (10800,)
test_class.shape: (1200,)


In [9]:
#Pacotes para rede neural
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [10]:
#Treinando o Tokenizador que irá transformar Palavras em Vetores
top_words = 1000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(dataframe['texto_preprocessing'])

print('\033[1m' + 'Dictionary size:' + '\033[0m', top_words)
print('\033[1m' + 'Length of the tokenizer index:' + '\033[0m', len(tokenizer.word_index))
print('\033[1m' + 'Number of documents the tokenizer was trained on:' + '\033[0m', tokenizer.document_count, '\n')
print('\033[1m' + 'First 20 entries of the tokenizer index:' + '\033[0m')
print(*list(tokenizer.word_index.items())[:20])

Dictionary size: 1000
Length of the tokenizer index: 7930
Number of documents the tokenizer was trained on: 12000 

First 20 entries of the tokenizer index:
('x000d', 1) ('ncia', 2) ('2024', 3) ('exposto', 4) ('recurso', 5) ('audi', 6) ('provimento', 7) ('julgo', 8) ('s', 9) ('art', 10) ('dia', 11) ('c', 12) ('processo', 13) ('designada', 14) ('ante', 15) ('concilia', 16) ('termos', 17) ('conhecer', 18) ('parte', 19) ('cpc', 20)


In [11]:
# Padronizando Tamanho dos textos
try:
  max_text_length = 100
  X_train = pad_sequences(X_train, maxlen=max_text_length)
  X_test = pad_sequences(X_test, maxlen=max_text_length)
except:
  pass

print('\033[1m' + 'X_train.shape:' + '\033[0m', X_train.shape)
print('\033[1m' + 'X_test.shape:' + '\033[0m', X_test.shape, '\n')

# Printing an example of review after padding

idx_pad = random.randint(0, len(X_train)-1)
print('\033[1m' + 'Review #%d after padding:' %idx_pad + '\033[0m' + '\n', X_train[idx_pad])

X_train.shape: (10800,)
X_test.shape: (1200,) 

Review #2340 after padding:
 unanimidade votos conhecer embargos declara


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

tf.compat.v1.reset_default_graph()

# Criar o modelo
model = Sequential()

# Adicionar camadas
model.add(Embedding(input_dim=100, output_dim=100))
model.add(LSTM(units=128))
model.add(Dense(units=10, activation='softmax'))

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Verificar o sumário do modelo
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)